In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


chat = ChatOpenAI(temperature=0.1)

poem_template = ChatPromptTemplate.from_messages([
    ("system","You are a poet skilled in writing poems about programming languages."),
    ("human", "Write a short, creative poem about the programming language: {language_name}"),
])

poem_chain = poem_template | chat

In [9]:
poem_chain.invoke({"language_name":"Python"})

AIMessage(content="In the land of code, where serpents slither,\nPython reigns, a language that's neither\nToo complex nor too simple, it strikes a balance,\nWith syntax clean, like a graceful dance.\n\nIts libraries vast, its power immense,\nFrom web apps to data science, it makes sense.\nIndentation matters, in Python's domain,\nFor readability and elegance, it's the main.\n\nSo embrace the snake, let it guide your hand,\nPython, the language that helps you stand\nAbove the rest, with its beauty and grace,\nA tool for creation in the coding race.")

In [5]:
explain_poem_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an AI that explains poems."),
    ("human","Explain the meaning and theme of this poem:\n\n{poem}"),
])

In [6]:
explain_chain = explain_poem_prompt | chat

In [7]:
final_chain = {"poem":poem_chain} | explain_chain

In [8]:
final_chain.invoke({
    "language_name":"python"
})

AIMessage(content="This poem is an ode to the programming language Python, using imagery of serpents and ruling over a land of code to convey its significance and versatility. The poet highlights Python's balance between complexity and simplicity, its clean syntax, and the importance of indentation for readability. The theme of the poem revolves around Python's dominance in various fields such as web development and data science, thanks to its versatility and the abundance of libraries like pandas and numpy that empower users to accomplish tasks more efficiently. The overall message celebrates Python as a powerful and essential tool in the world of programming.")